In [1]:
import pandas as pd
import numpy as np
import requests
import time
import psycopg2
import random
from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from Keyfile import key_fun #Custom file to hide postgreSQL Server information

In [2]:
key = key_fun()

engine = create_engine('postgresql://' + key['username'] + ':' + key['pwd'] + '@localhost:' + 
                       key['port_id'] + '/' + key['database'])

In [3]:
driver = webdriver.Chrome()
url = "https://www.nba.com/stats/players/traditional?PerMode=Totals&Season=2002-03&SeasonType=Regular%20Season&dir=A&sort=PLUS_MINUS"
driver.get(url)

In [4]:
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find("table", attrs = {"class" : "Crom_table__p1iZz"})
headers = table.findAll('th')
headerlist = [h.text.strip().upper() for h in headers if not 'RANK' in h.text.strip() and h.text.strip() in ['Player', 'Team', 'DD2','TD3', '+/-']]
header_index = {i: header for i, header in enumerate(headerlist)}
header_index

{0: 'PLAYER', 1: 'TEAM', 2: 'DD2', 3: 'TD3', 4: '+/-'}

In [5]:
dd_data = {header: [] for header in headerlist}
dd_data['SEASON'] = []

In [6]:
seasons = [str(x) + '-' + str(x+1)[2:] for x in range(1996,2023)]
for season in seasons:
    time.sleep( random.randint(1,4) + random.random() )
    
    url = "https://www.nba.com/stats/players/traditional?PerMode=Totals&Season=" + season + \
          "&SeasonType=Regular+Season&dir=A&sort=PLUS_MINUS"
    
    driver.get(url)
    
    while True:
        try:
            select = Select(driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
            break
        except:
            time.sleep( random.randint(1,8) + random.random() )
            driver.refresh()
        
    select.select_by_index(0)
    src = driver.page_source
    parser = BeautifulSoup(src, 'lxml')
    table = parser.find("table", attrs = {"class" : "Crom_table__p1iZz"})
    rows = table.findAll('td')
    stats = [row.text.strip() for i, row in enumerate(rows)  if i%30 in [1,2,27,28,29]]

    
    for i, stat in enumerate(stats):
        i = i%5
        dd_data[ header_index[i] ].append(stat)
        if i == 0:
            dd_data[ 'SEASON' ].append(season)

In [7]:
df_dd = pd.DataFrame.from_dict(dd_data)
df_dd.head()

,PLAYER,TEAM,DD2,TD3,+/-,SEASON
0,Michael Jordan,CHI,9,1,820,1996-97
1,Scottie Pippen,CHI,14,0,809,1996-97
2,Jeff Hornacek,UTA,1,0,775,1996-97
3,Karl Malone,UTA,43,1,767,1996-97
4,Bryon Russell,UTA,3,0,681,1996-97


In [8]:
df_dd.to_sql('double2_data', con = engine, if_exists = 'replace')

846